# Cette étude, via un algorithme de Machine Learning (classification), consiste à prévoir si un prospect est susceptible d'investir une somme d'argent dans un dépôt à terme.

# Le dataset a été récupéré depuis l'adresse suivante: 

https://archive.ics.uci.edu/ml/datasets/Bank+Marketing 

In [1]:
# Commençons par importer les premiers packages nécessaires:

%matplotlib inline
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [4]:
# Chargeons le dataset dans un dataframe nommé df:

df = pd.read_csv('bank.csv', sep = ';')

In [6]:
# Aperçu des 5 premières lignes du dataframe:

df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [10]:
# Affichage des principales informations propres à df:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


# Nous avons 17 variables, dont 16 explicatives et une variable cible à prédire (y).
# Il y'a 4521 lignes au total.

In [11]:
# Vérifions s'il y'a des valeurs nulles:

df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

# Il n'y a aucune valeur nulle dans df

# Le dataframe contient des variables quantitatives (int64) et qualitatives (object). 
# Comme nous allons faire tourner un algorithme de Forêts Aléatoires (Random Forest), il nous faut transformer les variables qualitatives en variables indicatrices.

In [25]:
df = df.join(pd.get_dummies(df['job'], prefix = 'job'))

In [27]:
df = df.join(pd.get_dummies(df['marital'], prefix = 'marital'))

In [28]:
df = df.join(pd.get_dummies(df['education'], prefix = 'education'))

In [29]:
df = df.join(pd.get_dummies(df['default'], prefix = 'default'))

In [30]:
df = df.join(pd.get_dummies(df['housing'], prefix = 'housing'))

In [31]:
df = df.join(pd.get_dummies(df['loan'], prefix = 'loan'))

In [32]:
df = df.join(pd.get_dummies(df['contact'], prefix = 'contact'))

In [33]:
df = df.join(pd.get_dummies(df['month'], prefix = 'month'))

In [34]:
df = df.join(pd.get_dummies(df['poutcome'], prefix = 'poutcome'))

# Isolons la variable cible (y) dans target:

In [35]:
target = df['y']

# Supprimons les colonnes desquelles nous avons obtenu les variables indicatrices, ainsi que la variable (y) de df, pour créer un autre dataframe nommé "data":

In [36]:
delete = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']

In [37]:
data = df.drop(delete, axis = 1)

In [39]:
# Visualisation des 5 premières lignes de "data":
data.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,30,1787,19,79,1,-1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,33,4789,11,220,1,339,4,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,35,1350,16,185,1,330,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,30,1476,3,199,4,-1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,59,0,5,226,1,-1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1


# Notre jeu de données est prêt pour le Machine Learning via l'algorithme "Random Forest".

# Tout d'abord, il nous faut séparer le jeu de donées et deux ensembles, d'entrainement et de test:

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, random_state = 126)

# Il est temps d'entrainer notre modèle:

In [41]:
# Importation du package nécessaire pour faire tourner l'algorithme:

from sklearn import ensemble

# Créer un classificateur rf:

In [42]:
rf = ensemble.RandomForestClassifier(n_jobs = -1, random_state = 321)
rf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=321)

# Passons à la prédiction des données de l'ensemble de test, en les stockant dans une variable nommée "y_pred":

In [43]:
y_pred = rf.predict(X_test)

# Affichons la matrice de confusion de l'échantillon de test: 

In [48]:
pd.crosstab(y_test, y_pred, rownames = ['Classe réelle'], colnames = ['Classe prédite'])

Classe prédite,no,yes
Classe réelle,,
no,803,9
yes,67,26


# Affichons le score de l'algorithme:

In [46]:
print(rf.score(X_test, y_test))

0.9160220994475138


In [49]:
df['y'].value_counts()

no     4000
yes     521
Name: y, dtype: int64

# Malgré un score élevé (91,6%), nous remarquons dans la matrice de confusion que la prédiction n'est pas optimale, notamment pour l'ocurrence "yes".
# Ceci est dû au fait que notre jeu de données est déséquilibré. En effet, il y'a 4000 "no" et seulement 521 "yes".

# Dans la prochaine étape, on va remédier à ce biais en équilibrant notre jeu de données, en lui appliquant cette fois un autre algorithme de Machine Learning, en l'occurrence le SVM (Support Vector Machine).